In [1]:
import json
with open('./tingyun.json', 'r') as f:
    datas = json.load(f)

In [2]:
from bs4 import BeautifulSoup
import html2text

In [59]:
def has_link(name, string):
    soup = BeautifulSoup(name, 'html.parser')
    text = [li.get_text(strip=True) for li in soup.find_all(string)]
    links = ['链接文本：{}，链接：{}'.format(li.find('a').get_text(),li.find('a').get('href')) for li in soup.find_all(string)]
    return text,links
has_link(datas[12]['解决问题及方案'], 'p')

(['可以按照 Doris 官网文档中的元数据运维文档进行恢复。'],
 ['链接文本：元数据运维文档，链接：https://cloud.tencent.com/document/product/1387/70722'])

In [37]:
def has_normal(name, string):
    soup = BeautifulSoup(name, 'html.parser')
    return [li.get_text(strip=True) for li in soup.find_all(string)]
has_normal(datas[11]['解决问题及方案'], 'li')

['尝试本地登录故障主机资源，或者登录同网段的其他资源进行测试；',
 '检查主机/etc/hosts.deny文件配置，是否将CBH系统IP加入了黑名单',
 '检查CBH系统与故障主机的网络层，是否有服务协议拦截CBH系统IP；',
 '请填写工单反馈问题现象，联系技术支持。']

In [3]:
def has_img(name, string):
    contexts = []
    soup = BeautifulSoup(name, 'html.parser')
    for li in soup.find_all(string):
        if li.get_text(strip=True):
            context = [li.get_text(strip=True)]
        else:
            context = []
        if  li.find('img'):
            links = li.find('img').get('src')
            context.append(links)
        if context:
            contexts.append(context)
    return contexts
has_img(datas[13]['原因分析'], 'p')

[['日志分析：下发上电动作时，存在发送上电请求信息给管理平台时，表明当前环境上电受控'],
 ['https://download.huawei.com/mdl/image/download?uuid=8c67c5afb31c495c9589d950f80810bc'],
 ['ipmcget -d poweronpermit查询受控状态'],
 ['https://download.huawei.com/mdl/image/download?uuid=9b5f498dcf364bb096d5f1416195f1d8']]

In [38]:
def has_img_links(name, string):
    contexts = []
    soup = BeautifulSoup(name, 'html.parser')
    for li in soup.find_all(string):
        if li.get_text(strip=True):
            context = [li.get_text(strip=True)]
        else:
            context = []
            
        if  li.find('img'):
            imgs = li.find('img').get('src')
            context.append(imgs)
            
        if li.find('a'):
            text = li.find('a').get_text()
            get_text = li.find('a').get('href')
            context.append('链接文本：{}，链接：{}'.format(text, get_text))
            
        if context:
            contexts.append(context)
    return contexts
has_img_links(datas[13]['解决问题及方案'], 'p')

[['法一：下发上电命令后再管理平台处理来自BMC的上电请求'],
 ['法二：先关闭受控上电，在执行上电动作，再恢复受控上电'],
 ['https://download.huawei.com/mdl/image/download?uuid=b04cb5039c5f4860b4a52ccb81c3ca7f']]

In [50]:
import re
pattern = r"</([^>]*)>"
for data in datas:
    for key,value in data.items():
        contexts = None
        if '<ol>' in value:  # 无序列表
            contexts = has_img_links(value, 'li')
        elif '<ul>' in value:  # 有序列表
            contexts = has_img_links(value, 'li')
        elif '<p>' in value:  # 段落
            contexts = has_img_links(value, 'p')
        
        if contexts:
            data[key] = contexts

html 语言转换成 markdown

In [7]:
import html2text
print(datas[12]['解决问题及方案'])
markdown_text = html2text.html2text(datas[12]['解决问题及方案'])
print(markdown_text)

<p>可以按照 Doris 官网文档中的&nbsp;<a href="https://cloud.tencent.com/document/product/1387/70722" target="_blank">元数据运维文档</a>&nbsp;进行恢复。<br/></p>
可以按照 Doris 官网文档中的
[元数据运维文档](https://cloud.tencent.com/document/product/1387/70722) 进行恢复。  




____________

单独写入 markdown

In [21]:
data = datas[11]
with open('output.md', 'w') as f:
    f.write('标题：{}\n'.format(html2text.html2text(data['标题'])))
    f.write('问题描述: {}\n'.format(html2text.html2text(data['问题描述'])))
    f.write('所选分组：{}\n'.format(html2text.html2text(data['所选分组'])))
    f.write('原因分析：\n{}\n'.format(html2text.html2text(data['原因分析'])))
    f.write('解决问题及方案：\n\n{}\n'.format(html2text.html2text(data['解决问题及方案'])))

每个数据写入不同的 markdown

In [3]:
for index, data in enumerate(datas):
    with open('./听云/{}.md'.format(index), 'w') as f:
        f.write('标题：{}\n'.format(html2text.html2text(data['标题'])))
        f.write('问题描述: {}\n'.format(html2text.html2text(data['问题描述'])))
        f.write('所选分组：{}\n'.format(html2text.html2text(data['所选分组'])))
        f.write('原因分析：\n{}\n'.format(html2text.html2text(data['原因分析'])))
        f.write('解决问题及方案：\n\n{}\n'.format(html2text.html2text(data['解决问题及方案'])))

所有数据写入同一个 markdown

In [6]:
with open('./听云/all.md', 'w') as f:
    for index, data in enumerate(datas):
        f.write('## index：{}\n'.format(index))
        f.write('标题：{}\n'.format(html2text.html2text(data['标题'])))
        f.write('问题描述: {}\n'.format(html2text.html2text(data['问题描述'])))
        f.write('所选分组：{}\n'.format(html2text.html2text(data['所选分组'])))
        f.write('原因分析：\n{}\n'.format(html2text.html2text(data['原因分析'])))
        f.write('解决问题及方案：\n\n{}\n'.format(html2text.html2text(data['解决问题及方案'])))